In [3]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, vectorize, stencil

# 4.2 Expectation Value

In [82]:
π = np.pi
exp = np.exp
step = 0.5

K = 10_000
dx_rand = np.random.rand(K)
metro_rand = np.random.rand(K)
accept = np.full(K, False)
x = np.full(K, np.nan)
x[0] = 0.0

for k in range(1, K):
    init = 0.5 * x[k - 1] ** 2
    dx = (dx_rand[k] - 0.5) * step * 2.0
    x[k] = x[k - 1] + dx
    fin = 0.5 * x[k] ** 2.0
    accept[k] = exp(init - fin) > metro_rand[k]
    x[k] = x[k] if accept[k] else x[k - 1]

accept[1:].sum() / K

0.8995